<a href="https://colab.research.google.com/github/shris2810/link_prediction/blob/main/GCN_and_GAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAT layer
* X = feature matrix of all node = [ [features_of_node_1] , [features_of_node_2] ]
* adjacency matrix who is connected to whom, NO normalization, NO degree information
* Linear transformation, Wh = XW
* Attention mechanism = attention score for each edge
  * e_ij ​= a ⋅ [ Wh_i ​∣∣ Wh_j ​]
  * GAT self-loop included as another neighbor for attention
* Softmax (normalize attention) = Each node normalizes only over its neighbors
  * alpha_ij = exp(e_ij) / ∑ (exp(e_ik)) , k is all neighbor nodes of i
* Weighted aggregation,
  * H_i = ∑ (alpha_ij * Wh_ij) , j is all neighbor nodes of i

# GAT multi-head
* Multi-head many attention mechanisms
* Each head has:
  * its own W
  * its own a
  * Then outputs are:
    * concatenated (hidden layers)
    * averaged (final layer)
* For the final output, we usually apply a linear layer (weight matrix) after convolutional layer to change the feature shape to what the task needs.

In [28]:
# @title
''' W1 = [[1, 0],
          [0, 1]]
    a1 = [1, 1, 1, 1]

    W2 = [[1],
          [1]]
    a2 = [1,1]
'''

X = {
    0: [1.0, 0.0],
    1: [0.0, 1.0],
    2: [1.0, 1.0]
}

adj = {
    0: [1, 2],
    1: [0, 2],
    2: [0, 1]
}

In [3]:
# @title
import math
import random

def leaky_relu(x, a=0.2):
    return x if x > 0 else a * x

def relu(x):
    return max(0, x)

def softmax(xs):
    exps = [math.exp(x) for x in xs]
    s = sum(exps)
    return [e / s for e in exps]


In [29]:
class GATHead:
    def __init__(self, in_dim, out_dim):
        self.W = [[random.uniform(-0.1, 0.1) for _ in range(out_dim)]
                  for _ in range(in_dim)]
        self.a = [random.uniform(-0.1, 0.1) for _ in range(2 * out_dim)]

    def linear(self, x):
        return [
            sum(x[i] * self.W[i][j] for i in range(len(x)))
            for j in range(len(self.W[0]))
        ]

    def forward(self, X, adj):
        H = {i: self.linear(X[i]) for i in X}
        Z = {}

        for i in H:
            scores = []
            neigh = adj[i]

            # e_ij ​= a ⋅ [ Wh_i ​∣∣ Wh_j ​]
            for j in neigh:
                concat = H[i] + H[j]
                e = sum(self.a[k] * concat[k] for k in range(len(concat)))
                scores.append(leaky_relu(e))

            # normalize attention
            alpha = softmax(scores)

            # out_i = [ ∑ (alpha_ij * Wh_ij[0])_0, ∑ (alpha_ij * Wh_ij[1])_1 ]
            out = [0.0] * len(H[i])
            for idx, j in enumerate(neigh):
                for d in range(len(out)):
                    out[d] += alpha[idx] * H[j][d]

            # each node has 2 feature (for our example)
            Z[i] = out

        return Z


In [5]:
class MultiHeadGAT:
    def __init__(self, in_dim, out_dim, heads=2):
        self.heads = [GATHead(in_dim, out_dim) for _ in range(heads)]

    def forward(self, X, adj):
        outputs = [head.forward(X, adj) for head in self.heads]
        Z = {}

        # each head gives 2 features (for our example)
        for node in X:
            Z[node] = []
            for out in outputs:
                Z[node] += out[node]  # concatenate

        # since we using 2 heads Z has 4 embeddings 2 for each head (for our example)
        return Z


In [6]:
class GATModel:
    def __init__(self):
        self.layer1 = MultiHeadGAT(2, 2, heads=2)
        self.layer2 = MultiHeadGAT(4, 2, heads=2)

    def forward(self, X, adj):
        h1 = self.layer1.forward(X, adj)
        h1 = {i: [relu(v) for v in h1[i]] for i in h1}
        h2 = self.layer2.forward(h1, adj)
        return h2


---

# GCN layer
* The weight matrix (W) is used to shape (change the dimension of) output features and to learn useful feature combinations.
* X = feature matrix of all node = [ [features_of_node_1] , [features_of_node_2] ]
* Edge list → adjacency matrix (A)
* Add self-loops, A = A + I
* self node is included in normalization math
* Degree matrix (D̂) = number of neighbors of each node
* normalization, A = (D̂^−1/2)*(A)*(D̂^−1/2)
* H = A​XW
  * AX = features aggregation
  * AXW = tranform aggregated feature





In [27]:
''' W1 = [[1, 0],
          [0, 1]]

    W2 = [[1],
          [1]]
'''

X = {
    0: [1.0, 0.0],
    1: [0.0, 1.0],
    2: [1.0, 1.0]
}

adj = {
    0: [1, 2],
    1: [0, 2],
    2: [0, 1]
}


In [14]:
import random
# activation function
def relu(x):
    return max(0, x)
#A=A+I
def add_self_loops(adj):
    new_adj = {}
    for i in adj:
        new_adj[i] = adj[i] + [i]
    return new_adj
#degree
def compute_degree(adj):
    D = {}
    for i in adj:
        D[i] = len(adj[i])
    return D

def norm_coeff(i, j, D):
    return 1 / math.sqrt(D[i] * D[j])


In [15]:
class NormalizedGCNLayer:
    def __init__(self, in_dim, out_dim):
        self.W = [[random.uniform(-0.1, 0.1) for _ in range(out_dim)]
                  for _ in range(in_dim)]

    def linear(self, x):
        return [
            sum(x[i] * self.W[i][j] for i in range(len(x)))
            for j in range(len(self.W[0]))
        ]

    def forward(self, X, adj):
        adj_hat = add_self_loops(adj)
        D = compute_degree(adj_hat)

        Z = {}
        # AXW
        for i in X:
            agg = [0.0] * len(X[i])
            # A(norm)X
            for j in adj_hat[i]:
                coeff = norm_coeff(i, j, D)
                for d in range(len(agg)):
                    agg[d] += coeff * X[j][d]
            # AW
            Z[i] = self.linear(agg)

        return Z

class GCNLayer:
    def __init__(self, in_dim, out_dim):
        # in_dim x out_dim
        self.W = [[random.uniform(-0.1, 0.1) for _ in range(out_dim)]
                  for _ in range(in_dim)]

    def linear(self, x):
        # 1 x in_dim * in_dim x out_dim = 1 x out_dim
        return [
            sum(x[i] * self.W[i][j] for i in range(len(x)))
            for j in range(len(self.W[0]))
        ]

    def forward(self, X, adj):
        Z = {}

        for i in X:
            # add self-loop
            neighbors = adj[i] + [i]
            agg = [0.0] * len(X[i])

            for j in neighbors:
                for d in range(len(agg)):
                    agg[d] += X[j][d]

            # mean aggregation
            agg = [v / len(neighbors) for v in agg]

            # linear transform
            Z[i] = self.linear(agg)

        return Z


In [16]:
class TwoHopNormalizedGCN:
    def __init__(self):
        self.gcn1 = NormalizedGCNLayer(2, 4)
        self.gcn2 = NormalizedGCNLayer(4, 2)

    def forward(self, X, adj):
        h1 = self.gcn1.forward(X, adj)
        h1 = {i: [relu(v) for v in h1[i]] for i in h1}
        h2 = self.gcn2.forward(h1, adj)
        return h2

class TwoHopGCN:
    def __init__(self):
        self.gcn1 = GCNLayer(2, 4)
        self.gcn2 = GCNLayer(4, 2)

    def forward(self, X, adj):
        h1 = self.gcn1.forward(X, adj)
        h1 = {i: [relu(v) for v in h1[i]] for i in h1}
        h2 = self.gcn2.forward(h1, adj)
        return h2


---
# Running Models


In [25]:
model1 = TwoHopNormalizedGCN()
embeddings1 = model1.forward(X, adj)

for node, emb in embeddings1.items():
    print(f"Node {node}: {emb}")


Node 0: [0.002631944631269307, -0.00043283505612194944]
Node 1: [0.002631944631269307, -0.00043283505612194944]
Node 2: [0.002631944631269307, -0.00043283505612194944]


In [26]:
model2 = TwoHopGCN()
embeddings2 = model2.forward(X, adj)

for node, emb in embeddings2.items():
    print(f"Node {node}: {emb}")


Node 0: [-0.0015911831004312268, 0.0030330481138056632]
Node 1: [-0.0015911831004312268, 0.0030330481138056632]
Node 2: [-0.0015911831004312268, 0.0030330481138056632]


In [24]:
model3 = GATModel()
embeddings3 = model3.forward(X, adj)

for node, emb in embeddings3.items():
    print(f"Node {node}: {emb}")

Node 0: [-0.0031367278233775954, 0.000401440247721624, -0.00251948564280926, -0.001431605938432978]
Node 1: [-0.004020932591701411, 0.00029792094977793915, -0.002811547994734662, -0.0011620188745751258]
Node 2: [-0.004292621699663606, 0.0004196267219622892, -0.00319750177297253, -0.001556037419797711]
